In [12]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdDetermineBonds
from rdkit.Chem.Draw import IPythonConsole
from openff.toolkit import Molecule, Topology, ForceField
from openff.interchange import Interchange
from openff.units import unit
from openff.units.openmm import from_openmm, to_openmm
import openmm
import numpy as np
import sys
from pdb_wizard import PBC

In [2]:
mol = Chem.MolFromXYZFile('./HKUST-1.xyz')

rdDetermineBonds.DetermineConnectivity(mol)
IPythonConsole.drawMol3D(mol)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [3]:
editable = Chem.EditableMol(mol)
# No kill like over kill
METALS = [3, 4, 11, 12, 13, 19, 20, 21, 22, 23, 24, 25 ,26, 27, 28, 29, 30, 31, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]

for bond in mol.GetBonds():
    a1 = bond.GetBeginAtom()
    a1_idx = bond.GetBeginAtomIdx()
    a2 = bond.GetEndAtom()
    a2_idx = bond.GetEndAtomIdx()
    if a1.GetAtomicNum() in METALS or a2.GetAtomicNum() in METALS:
        editable.RemoveBond(a1_idx, a2_idx)
    
IPythonConsole.drawMol3D(editable.GetMol())

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [7]:
# You could change sanitizeFrags to True, but I doubt it's necessary
frags = Chem.GetMolFrags(editable.GetMol(), asMols=True, sanitizeFrags=True)
for frag in frags:
    if frag.GetNumAtoms() == 1:
        charge = 2
    else:
        charge = -3
    rdDetermineBonds.DetermineBonds(frag, charge=charge)

In [8]:
mols = [Molecule.from_rdkit(frag) for frag in frags]

In [18]:
for mol in mols:
    if mol.to_smiles() == '[Cu]':
        mol.atom(0).formal_charge = 2

In [ ]:
def append_system(existing_system, system_to_append, cutoff, index_map=None):
    """Appends a system object onto the end of an existing system.

    Parameters
    ----------
    existing_system: openmm.System, optional
        The base system to extend.
    system_to_append: openmm.System
        The system to append.
    cutoff: openff.evaluator.unit.Quantity
        The nonbonded cutoff
    index_map: dict of int and int, optional
        A map to apply to the indices of atoms in the `system_to_append`.
        This is predominantly to be used when the ordering of the atoms
        in the `system_to_append` does not match the ordering in the full
        topology.
    """
    supported_force_types = [
        openmm.HarmonicBondForce,
        openmm.HarmonicAngleForce,
        openmm.PeriodicTorsionForce,
        openmm.NonbondedForce,
        openmm.RBTorsionForce,
        openmm.CustomNonbondedForce,
        openmm.CustomBondForce,
    ]

    number_of_appended_forces = 0
    index_offset = existing_system.getNumParticles()

    # Create an index map if one is not provided.
    if index_map is None:
        index_map = {i: i for i in range(system_to_append.getNumParticles())}

    # Append the particles.
    for index in range(system_to_append.getNumParticles()):
        index = index_map[index]
        existing_system.addParticle(system_to_append.getParticleMass(index))

    # Append the constraints
    for index in range(system_to_append.getNumConstraints()):
        index_a, index_b, distance = system_to_append.getConstraintParameters(index)

        index_a = index_map[index_a]
        index_b = index_map[index_b]

        existing_system.addConstraint(
            index_a + index_offset, index_b + index_offset, distance
        )

    # Validate the forces to append.
    for force_to_append in system_to_append.getForces():
        if type(force_to_append) in supported_force_types:
            continue

        raise ValueError(
            f"The system contains an unsupported type of "
            f"force: {type(force_to_append)}."
        )

    # Append the forces.
    for force_to_append in system_to_append.getForces():
        existing_force = None

        for force in existing_system.getForces():
            if type(force) not in supported_force_types:
                raise ValueError(
                    f"The existing system contains an unsupported type "
                    f"of force: {type(force)}."
                )

            if type(force_to_append) is not type(force):
                continue

            if isinstance(
                force_to_append, openmm.CustomNonbondedForce
            ) or isinstance(force_to_append, openmm.CustomBondForce):
                if force_to_append.getEnergyFunction() != force.getEnergyFunction():
                    continue

            existing_force = force
            break

        if existing_force is None:
            if isinstance(force_to_append, openmm.CustomNonbondedForce):
                existing_force = openmm.CustomNonbondedForce(
                    force_to_append.getEnergyFunction()
                )
                existing_force.setCutoffDistance(cutoff)
                existing_force.setNonbondedMethod(
                    openmm.CustomNonbondedForce.CutoffPeriodic
                )
                for index in range(force_to_append.getNumGlobalParameters()):
                    existing_force.addGlobalParameter(
                        force_to_append.getGlobalParameterName(index),
                        force_to_append.getGlobalParameterDefaultValue(index),
                    )
                for index in range(force_to_append.getNumPerParticleParameters()):
                    existing_force.addPerParticleParameter(
                        force_to_append.getPerParticleParameterName(index)
                    )
                existing_system.addForce(existing_force)

            elif isinstance(force_to_append, openmm.CustomBondForce):
                existing_force = openmm.CustomBondForce(
                    force_to_append.getEnergyFunction()
                )
                for index in range(force_to_append.getNumGlobalParameters()):
                    existing_force.addGlobalParameter(
                        force_to_append.getGlobalParameterName(index),
                        force_to_append.getGlobalParameterDefaultValue(index),
                    )
                for index in range(force_to_append.getNumPerBondParameters()):
                    existing_force.addPerBondParameter(
                        force_to_append.getPerBondParameterName(index)
                    )
                existing_system.addForce(existing_force)

            else:
                existing_force = type(force_to_append)()
                existing_system.addForce(existing_force)

        if isinstance(force_to_append, openmm.HarmonicBondForce):
            # Add the bonds.
            for index in range(force_to_append.getNumBonds()):
                index_a, index_b, *parameters = force_to_append.getBondParameters(
                    index
                )

                index_a = index_map[index_a]
                index_b = index_map[index_b]

                existing_force.addBond(
                    index_a + index_offset, index_b + index_offset, *parameters
                )

        elif isinstance(force_to_append, openmm.HarmonicAngleForce):
            # Add the angles.
            for index in range(force_to_append.getNumAngles()):
                (
                    index_a,
                    index_b,
                    index_c,
                    *parameters,
                ) = force_to_append.getAngleParameters(index)

                index_a = index_map[index_a]
                index_b = index_map[index_b]
                index_c = index_map[index_c]

                existing_force.addAngle(
                    index_a + index_offset,
                    index_b + index_offset,
                    index_c + index_offset,
                    *parameters,
                )

        elif isinstance(force_to_append, openmm.PeriodicTorsionForce):
            # Add the torsions.
            for index in range(force_to_append.getNumTorsions()):
                (
                    index_a,
                    index_b,
                    index_c,
                    index_d,
                    *parameters,
                ) = force_to_append.getTorsionParameters(index)

                index_a = index_map[index_a]
                index_b = index_map[index_b]
                index_c = index_map[index_c]
                index_d = index_map[index_d]

                existing_force.addTorsion(
                    index_a + index_offset,
                    index_b + index_offset,
                    index_c + index_offset,
                    index_d + index_offset,
                    *parameters,
                )

        elif isinstance(force_to_append, openmm.NonbondedForce):
            # Add the vdW parameters
            for index in range(force_to_append.getNumParticles()):
                index = index_map[index]

                existing_force.addParticle(
                    *force_to_append.getParticleParameters(index)
                )

            # Add the 1-2, 1-3 and 1-4 exceptions.
            for index in range(force_to_append.getNumExceptions()):
                (
                    index_a,
                    index_b,
                    *parameters,
                ) = force_to_append.getExceptionParameters(index)

                index_a = index_map[index_a]
                index_b = index_map[index_b]

                existing_force.addException(
                    index_a + index_offset, index_b + index_offset, *parameters
                )

        elif isinstance(force_to_append, openmm.RBTorsionForce):
            # Support for RBTorisionForce needed for OPLSAA, etc
            for index in range(force_to_append.getNumTorsions()):
                torsion_params = force_to_append.getTorsionParameters(index)
                for i in range(4):
                    torsion_params[i] = index_map[torsion_params[i]] + index_offset

                existing_force.addTorsion(*torsion_params)

        elif isinstance(force_to_append, openmm.CustomNonbondedForce):
            for index in range(force_to_append.getNumParticles()):
                nb_params = force_to_append.getParticleParameters(index_map[index])
                existing_force.addParticle(nb_params)

            # Add the 1-2, 1-3 and 1-4 exceptions.
            for index in range(force_to_append.getNumExclusions()):
                (
                    index_a,
                    index_b,
                ) = force_to_append.getExclusionParticles(index)

                index_a = index_map[index_a]
                index_b = index_map[index_b]

                existing_force.addExclusion(
                    index_a + index_offset, index_b + index_offset
                )

        elif isinstance(force_to_append, openmm.CustomBondForce):
            for index in range(force_to_append.getNumBonds()):
                index_a, index_b, bond_params = force_to_append.getBondParameters(
                    index
                )

                index_a = index_map[index_a] + index_offset
                index_b = index_map[index_b] + index_offset

                existing_force.addBond(index_a, index_b, bond_params)

        number_of_appended_forces += 1

    if number_of_appended_forces != system_to_append.getNumForces():
        raise ValueError("Not all forces were appended.")

def create_empty_system():
    system = openmm.System()
    system.addForce(openmm.HarmonicBondForce())
    system.addForce(openmm.HarmonicAngleForce())
    system.addForce(openmm.PeriodicTorsionForce())

    nonbonded_force = openmm.NonbondedForce()
    nonbonded_force.setNonbondedMethod(openmm.NonbondedForce.NoCutoff)

    system.addForce(nonbonded_force)

    return system

In [ ]:
TEMP = to_openmm(298 * unit.kelvin)
PRESSURE = to_openmm(1 * unit.atm)
TIMESTEP = to_openmm(2 * unit.femtosecond)
CO2 = Molecule.from_smiles('O=C=O')

In [19]:
top = Topology.from_molecules(mols)
top.box_vectors = np.array([[26.343, 0, 0], [0, 26.343, 0], [0, 0, 26.343]]) * unit.angstrom
top.is_periodic = True

openff_ff = ForceField("openff-2.2.0-uff.offxml", load_plugins=True)
interchange = Interchange.from_smirnoff(topology=top, force_field=openff_ff)
openmm_sys = interchange.to_openmm(combine_nonbonded_forces=False)

openmm_sys.addForce(openmm.MonteCarloBarostat(PRESSURE, TEMP))
openmm_integrator = openmm.LangevinIntegrator(TEMP, 1, TIMESTEP)

openmm_top = top.to_openmm()
openmm_sim = openmm.app.Simulation(openmm_top, openmm_sys, openmm_integrator, platform=openmm.Platform.getPlatformByName("CPU"))
openmm_sim.f
reporter = openmm.app.StateDataReporter(sys.stdout, 1000, step=True, time=True, potentialEnergy=True,
                                        kineticEnergy=True, totalEnergy=True, temperature=True, speed=True, density=True)
openmm_sim.reporters.append(reporter)
openmm_sim.reporters.append(openmm.app.PDBReporter('out.pdb', 1))
openmm_sim.context.setPositions(to_openmm(top.get_positions()))
openmm_sim.context.setVelocitiesToTemperature(TEMP)
openmm_sim.minimizeEnergy()
openmm_sim.step(10000)


#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Density (g/mL)","Speed (ns/day)"
1000,2.0000000000000013,-109458.5571034122,2129.1042220128443,-107329.45288139935,288.36979607308086,0.8560488693521695,0
2000,3.999999999999781,-109582.5876407017,2164.2319296441237,-107418.35571105758,293.12755747405646,0.8460322469479905,13.8
3000,5.999999999999561,-109708.29183840929,2303.3419331965106,-107404.94990521278,311.96887249347935,0.8381412490170187,13.7
4000,7.999999999999341,-109792.42943180335,2206.592735416398,-107585.83669638695,298.864983005294,0.8400921005965387,13.7
5000,10.000000000000009,-109789.94419829677,2147.032192698236,-107642.91200559854,290.79799343284327,0.8376847294834661,13.8
6000,12.000000000000677,-109749.0407964767,2278.7104974078648,-107470.33029906882,308.63274547728105,0.8467137476129993,13.8
7000,14.000000000001345,-109579.63071822861,2311.3757982420766,-107268.25491998653,313.05699396771996,0.